# Using Non Earth dataset

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~= 3.0
- ipyleaflet

In [ ]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [ ]:
import requests
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections
)

In [ ]:
# For this DEMO we will use this file
src_path = "https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/galileo_voyager/controlled_mosaics/11ESCOLORS01-02_GalileoSSI_Equi-cog.tif"

## Tile Server

For this demo, we need to create a minimal tile server.

In [ ]:
import json
from concurrent import futures

from tornado import web
from tornado import gen
from tornado.httpserver import HTTPServer
from tornado.concurrent import run_on_executor

from rio_tiler.io import COGReader
from rio_tiler.errors import TileOutsideBounds
from rio_tiler.profiles import img_profiles

from pyproj import CRS
from morecantile import TileMatrixSet

# Create a CUSTOM TMS using the europa ESRI:104915 projection
europa_crs = CRS.from_authority("ESRI", 104915)
europa_tms = TileMatrixSet.custom(
    crs=europa_crs, extent=europa_crs.area_of_use.bounds, matrix_scale=[2, 1],
)

class TileServer:
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application([
            (r"^/tiles/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
        ])

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)
  
    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, z, x, y):

        try:
            with COGReader(self.url, tms=europa_tms) as cog:
                data = cog.tile(x, y, z)
        except TileOutsideBounds:
            raise web.HTTPError(404)

        image = data.post_process(in_range=((0, 0.5),))

        prof = img_profiles.get("PNG", {})
        return image.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

In [ ]:
bounds = (129.36834223297478, 13.985559117409744, 138.90253908503576, 23.13673177454536)

m = Map(
    center=(
        (bounds[1] + bounds[3]) / 2,
        (bounds[0] + bounds[2]) / 2
    ),
    zoom=4,
    basemap={},
    crs=projections.EPSG4326,  # HACK: the europa TMS is in degree and covers -180, -90, 180, 90 like the WGS84
)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/{z}/{x}/{y}",
    min_zoom=4,
    max_zoom=6,
    opacity=1,
)
m.add_layer(layer)
m

In [ ]:
ts.stop()